In [2]:
from pyspark.sql import SQLContext, Row
import operator

print sc.defaultParallelism
print sc.defaultMinPartitions
sqlContext = SQLContext(sc)
tmp = sqlContext.read.format('jdbc').options(
        url="jdbc:mysql://localhost:3306/papercrawler?user=root&password=1234",
        dbtable="(select max(id) as maxId from manuscript) tmp",
        driver="com.mysql.jdbc.Driver"
).load()
maxId=tmp.select("maxId").collect()[0][0]
print maxId

df = sqlContext.read.format('jdbc').options(
        url='jdbc:mysql://localhost:3306/papercrawler?user=root&password=1234&zeroDateTimeBehavior=convertToNull&characterEncoding=UTF-8&useUnicode=true&mysqlEncoding=utf8&allowMultiQueries=true',
        dbtable='manuscript',
        partitionColumn="id",
        lowerBound = 1,
        upperBound = maxId,
        numPartitions = 100
).load()

print df.count()

print df.show()

4
2
5523031
3554993
+---+--------------------+--------------------+--------------------+--------+--------------+-----+----+---------+-------+------+-----+--------------------+----+--------+--------------+--------------------+----------------+--------------------+-----------+--------------------+
| id|                 doi|             dblpKey|               title|subtitle|manuscriptType|month|year|pageStart|pageEnd|volume|issue|        externalLink|note|metaSave|metaSaveSource|   metaSaveTimestamp|metaUpdateSource| metaUpdateTimestamp|archiveSave|archiveSaveTimestamp|
+---+--------------------+--------------------+--------------------+--------+--------------+-----+----+---------+-------+------+-----+--------------------+----+--------+--------------+--------------------+----------------+--------------------+-----------+--------------------+
|  1|10.1007/s11227-01...|journals/tjs/KimH...|Lifetime maximiza...|    null|       article| null|2013|      365|    382|    65|    1|http://dx.doi.o

In [3]:
name_rdd = df.select('title').rdd

print name_rdd.getNumPartitions()

name_s_rdd = name_rdd.map(lambda x: x.asDict()).map(lambda x: x['title']).filter(lambda x: x)
name_s_rdd.getNumPartitions()
words = name_s_rdd.flatMap(lambda x: x.split(" "))

def sort1():
    results = words.countByValue()
    sorted_results = sorted(results.items(), key=operator.itemgetter(1), reverse=True)


def sort2():
    results = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
    sorted_results = results.sortBy(lambda (x, y): y, ascending=False)

%timeit sort1()

%timeit sort2()

#results = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
#print results
#sorted_results = results.sortBy(lambda (x, y): y, ascending=False)
#for i in sorted_results.collect():
#    print i

100
1 loops, best of 3: 31.2 s per loop
1 loops, best of 3: 45.8 s per loop


In [9]:
results = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
print results
sorted_results = results.sortBy(lambda (x, y): y, ascending=False)

PythonRDD[130] at RDD at PythonRDD.scala:43


In [ ]:
for i in sorted_results.collect():
    print i

(u'of', 1297479)
(u'for', 1105290)
(u'and', 874136)
(u'in', 718316)
(u'the', 571756)
(u'A', 488897)
(u'a', 376477)
(u'on', 323725)
(u'with', 315979)
(u'to', 269573)
(u'The', 155721)
(u'An', 148158)
(u'using', 137094)
(u'Using', 133595)
(u'by', 101546)
(u'Based', 95017)
(u'based', 93927)
(u'an', 90762)
(u'On', 87962)
(u'from', 85797)
(u'-', 73677)
(u'Analysis', 73359)
(u'Data', 70192)
(u'Networks.', 62010)
(u'System', 61049)
(u'Systems.', 59379)
(u'Design', 58677)
(u'Model', 57212)
(u'Information', 51935)
(u'analysis', 50957)
(u'Learning', 49448)
(u'networks.', 48965)
(u'Approach', 46651)
(u'Algorithm', 46520)
(u'systems.', 45608)
(u'Performance', 45570)
(u'control', 43849)
(u'system', 43471)
(u'Distributed', 42592)
(u'Control', 42562)
(u'Efficient', 42475)
(u'model', 41800)
(u'Network', 41799)
(u'data', 40549)
(u'Systems', 39765)
(u'Dynamic', 39036)
(u'New', 38881)
(u'approach', 37823)
(u'algorithm', 37117)
(u'Wireless', 36830)
(u'Mobile', 36818)
(u'Adaptive', 36598)
(u'Image', 35885)


In [1]:
import tensorflow as tf

# Create a Constant op that produces a 1x2 matrix.  The op is
# added as a node to the default graph.
#
# The value returned by the constructor represents the output
# of the Constant op.
matrix1 = tf.constant([[3., 3.]])

# Create another Constant that produces a 2x1 matrix.
matrix2 = tf.constant([[2.],[2.]])

# Create a Matmul op that takes 'matrix1' and 'matrix2' as inputs.
# The returned value, 'product', represents the result of the matrix
# multiplication.
product = tf.matmul(matrix1, matrix2)

In [2]:
# Launch the default graph.
sess = tf.Session()

# To run the matmul op we call the session 'run()' method, passing 'product'
# which represents the output of the matmul op.  This indicates to the call
# that we want to get the output of the matmul op back.
#
# All inputs needed by the op are run automatically by the session.  They
# typically are run in parallel.
#
# The call 'run(product)' thus causes the execution of three ops in the
# graph: the two constants and matmul.
#
# The output of the op is returned in 'result' as a numpy `ndarray` object.
result = sess.run(product)
print(result)
# ==> [[ 12.]]

# Close the Session when we're done.
sess.close()

[[ 12.]]


In [3]:
with tf.Session() as sess:
  result = sess.run([product])
  print(result)

[array([[ 12.]], dtype=float32)]


In [5]:
import urllib2   
import tensorflow as tf  
import numpy as np  
import findspark     
 
findspark.init()  
from pyspark import SparkContext, SparkFiles, SQLContext
 
if not 'sc' in locals():  
    sc = SparkContext()  
 
if not 'sqlContext' in locals():
 sqlContext = SQLContext(sc)
 
wordsDF = sqlContext.createDataFrame([('cat',), ('elephant',), ('rat',), ('rat',), ('cat', )], ['word'])   
wordCountsDF = wordsDF.groupBy("word").count()   
wordCountsDF.show() 
 
x_data = np.random.rand(100).astype(np.float32)  
y_data = x_data * 0.1 + 0.3  
 
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))  
b = tf.Variable(tf.zeros([1]))  
y = W * x_data + b  
 
loss = tf.reduce_mean(tf.square(y - y_data))  
optimizer = tf.train.GradientDescentOptimizer(0.5)  
train = optimizer.minimize(loss)  
 
init = tf.initialize_all_variables()  
 
sess = tf.Session()  
sess.run(init)  
 
for step in range(201):  
    sess.run(train)  
    if step % 20 == 0:  
        print(step, sess.run(W), sess.run(b))

+--------+-----+
|    word|count|
+--------+-----+
|     rat|    2|
|     cat|    2|
|elephant|    1|
+--------+-----+

(0, array([-0.39185727], dtype=float32), array([ 0.7423085], dtype=float32))
(20, array([-0.04136314], dtype=float32), array([ 0.37251282], dtype=float32))
(40, array([ 0.06488077], dtype=float32), array([ 0.31801456], dtype=float32))
(60, array([ 0.09127525], dtype=float32), array([ 0.3044754], dtype=float32))
(80, array([ 0.09783248], dtype=float32), array([ 0.30111185], dtype=float32))
(100, array([ 0.09946152], dtype=float32), array([ 0.30027622], dtype=float32))
(120, array([ 0.09986621], dtype=float32), array([ 0.30006865], dtype=float32))
(140, array([ 0.09996674], dtype=float32), array([ 0.30001706], dtype=float32))
(160, array([ 0.09999175], dtype=float32), array([ 0.30000424], dtype=float32))
(180, array([ 0.09999794], dtype=float32), array([ 0.30000108], dtype=float32))
(200, array([ 0.0999995], dtype=float32), array([ 0.30000028], dtype=float32))
